In [1]:
from client import Chartmetric
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
from config import chartmetric_token
from config import refresh_command
import time
from tqdm import tqdm

In [9]:
def refresh_token():
    os.system(refresh_command)
refresh_token()

# cm = Chartmetric(chartmetric_token)
# df = pd.read_csv('tracks.csv')
# tests = df.loc[df.ISRC.notnull()]

# test_100 = tests[:100]
# test_200 = tests[100:200]
# test_300 = tests[200:300]
# test_400 = tests[300:400]
# test_500 = tests[400:500]
# test_600 = tests[500:600]
# test_700 = tests[600:700]

In [8]:
isrc_dict = {}
searches = {}
for test in tqdm(test_500.ISRC):
    searches[test] = cm.search(test.strip())
    time.sleep(4)
    
import pickle
with open('500.pickle', 'wb') as handle:
    pickle.dump(searches, handle, protocol=pickle.HIGHEST_PROTOCOL)


100%|██████████| 12/12 [01:30<00:00,  7.46s/it]

In [14]:
def get_cm_id(search_dict):
    try:
        cm_id = search_results['obj']['tracks'][0]['id']
        name_of_song = search_results["obj"]["tracks"][0]["name"]
        isrc_dict[name_of_song] = dict(cm_id=cm_id, isrc=test)
        time.sleep(5)
    except TypeError:
        print(f'{test} not found')
        time.sleep(5)

In [ ]:
def get_song_info(search_dict):
    for k, v in tqdm(isrc_dict.items()):
        song_data_dict[k] = cm.track(str(v['cm_id']))
        time.sleep(2)

In [25]:
song_data_dict = {}
for k, v in tqdm(isrc_dict.items()):
    song_data_dict[k] = cm.track(str(v['cm_id']))
    time.sleep()





  0%|          | 0/62 [00:00<?, ?it/s]



  2%|▏         | 1/62 [00:03<03:47,  3.72s/it]



  3%|▎         | 2/62 [00:07<03:44,  3.74s/it]



  5%|▍         | 3/62 [00:12<03:59,  4.07s/it]



  6%|▋         | 4/62 [00:15<03:47,  3.92s/it]



  8%|▊         | 5/62 [00:19<03:39,  3.85s/it]



 10%|▉         | 6/62 [00:23<03:31,  3.78s/it]



 11%|█▏        | 7/62 [00:26<03:25,  3.74s/it]

KeyboardInterrupt: 

In [169]:
song_data_dict

{'Ms. Jackson': {'obj': {'id': 15048628,
   'name': 'Ms. Jackson',
   'isrc': 'USAR10100783',
   'image_url': 'https://i.scdn.co/image/95498205e9169ea726cc7b27d7bffa66e05e41fa',
   'duration_ms': 239360,
   'composer_name': 'André Benjamin, Antwan Patton & David Sheats',
   'artists': [{'id': 180,
     'name': 'OutKast',
     'mbid': '73fdb566-a9b1-494c-9f32-51768ec9fd27',
     'image_url': None,
     'cover_url': None,
     'code2': 'US',
     'gender': 1,
     'isni': None,
     'description': None,
     'created_at': '2016-11-26',
     'hometown_city': None,
     'current_city': None,
     'date_of_birth': '1991-01-01',
     'date_of_death': None}],
   'albums': [{'id': 306001,
     'name': 'Big Boi & Dre Present, Outkast',
     'upc': '888880763624',
     'release_date': '2001-10-04T00:00:00.000Z',
     'label': 'Arista/LaFace Records',
     'image_url': 'https://i.scdn.co/image/95498205e9169ea726cc7b27d7bffa66e05e41fa',
     'popularity': 0},
    {'id': 1189534,
     'name': 'Big 

In [170]:
song_features_dict = {} 
for k, v in song_data_dict.items():
    song_features_dict[k] = v['obj']['cm_audio_features']
feature_df = pd.DataFrame.from_dict(song_features_dict).T
feature_df.drop(columns=["key", "loudness", "mode", "tempo"], inplace=True)

In [171]:
def sim_function(x, y):
    return 1. / (1. + np.linalg.norm(x - y, 2))

In [172]:
n = len(feature_df)
sim_song = {}
X = np.zeros((n, 7))

for i in range(n):
    X[i] = feature_df.iloc[i]
    
for i in range(n):
    x = feature_df.iloc[i]
    sim_song[x.name] = 0
    buffer = 0
    for j in range(n):
        if i == j:
            continue
            
        y = feature_df.iloc[j]
        
        sim = sim_function(x, y)
        if sim > buffer:
            sim_song[x.name] = y.name
            buffer = sim

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)

z = pca.transform(X)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7 * 1.6, 7))
for i, vec in enumerate(z):
    ax.scatter(vec[0], vec[1], label=feature_df.iloc[i].name)
plt.legend(loc="best", ncol=5, bbox_to_anchor=(0, 1.02, 1, 0.102),
          frameon=False, borderpad=2)
plt.show()